# Learn about DELTA tables

Microsoft Fabric Lakehouse and the Apache Spark engine support all table types, both managed and unmanaged; this includes views and regular non-Delta Hive table formats. Tables defined using PARQUET, CSV, AVRO, JSON, and any Apache Hive compatible file format work as expected.

Delta Lake is an open-source storage layer that brings ACID (atomicity, consistency, isolation, and durability) transactions to Apache Spark and big data workloads.

The current version of Delta Lake included with Azure Synapse has language support for Scala, PySpark, and .NET and is compatible with Linux Foundation Delta Lake. 

## ➡️ Key features

| Feature | Description |
| --- | --- |
| **ACID Transactions** | Data lakes are typically populated through multiple processes and pipelines, some of which are writing data concurrently with reads. Prior to Delta Lake and the addition of transactions, data engineers had to go through a manual error prone process to ensure data integrity. Delta Lake brings familiar ACID transactions to data lakes. It provides serializability, the strongest level of isolation level. Learn more at [Diving into Delta Lake: Unpacking the Transaction Log](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html).|
| **Scalable Metadata Handling** | In big data, even the metadata itself can be "big data." Delta Lake treats metadata just like data, leveraging Spark's distributed processing power to handle all its metadata. As a result, Delta Lake can handle petabyte-scale tables with billions of partitions and files at ease. |
| **Time Travel (data versioning)** | The ability to "undo" a change or go back to a previous version is one of the key features of transactions. Delta Lake provides snapshots of data enabling you to revert to earlier versions of data for audits, rollbacks or to reproduce experiments. Learn more in [Introducing Delta Lake Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html). |
| **Open Format** | Apache Parquet is the baseline format for Delta Lake, enabling you to leverage the efficient compression and encoding schemes that are native to the format. |
| **Unified Batch and Streaming Source and Sink** | A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box. |
| **Schema Enforcement** | Schema enforcement helps ensure that the data types are correct and required columns are present, preventing bad data from causing data inconsistency. For more information, see [Diving Into Delta Lake: Schema Enforcement & Evolution](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html) |
| **Schema Evolution** | Delta Lake enables you to make changes to a table schema that can be applied automatically, without having to write migration DDL. For more information, see [Diving Into Delta Lake: Schema Enforcement & Evolution](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html) |
| **Audit History** | Delta Lake transaction log records details about every change made to data providing a full audit trail of the changes. |
| **Updates and Deletes** | Delta Lake supports Scala / Java / Python and SQL APIs for a variety of functionality. Support for merge, update, and delete operations helps you to meet compliance requirements. For more information, see [Announcing the Delta Lake 0.6.1 Release](https://github.com/delta-io/delta/releases/tag/v0.6.1),  [Announcing the Delta Lake 0.7 Release](https://github.com/delta-io/delta/releases/tag/v0.7.0) and [Simple, Reliable Upserts and Deletes on Delta Lake Tables using Python APIs](https://databricks.com/blog/2019/10/03/simple-reliable-upserts-and-deletes-on-delta-lake-tables-using-python-apis.html), which includes code snippets for merge, update, and delete DML commands. |
| **100 percent compatible with Apache Spark API** | Developers can use Delta Lake with their existing data pipelines with minimal change as it is fully compatible with existing Spark implementations. |

For full documentation, see the [Delta Lake Documentation Page](https://docs.delta.io/latest/delta-intro.html)


Let's get started by displaying the delta version. For more information about the delta lake releases, see [here](https://docs.delta.io/latest/releases.html)


In [1]:
%%spark

println(io.delta.VERSION)

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 3, Finished, Available)

2.4.0.8


Next, we want to have a look at the default data source to use for the DataFrameReader or DataFrameWriter within Microsoft Fabric. We can confirm that delta is the default.

In [2]:
%%sql
SET spark.sql.sources.default

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 4, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

## ➡️ DROP and CREATE TABLE

If the delta101 table already exists (check your lakehouse), drop it. This way, we can repeatedly execute this notebook without having to clean up files manually.


In [3]:
%%sql
DROP TABLE IF EXISTS delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 5, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

Now, let's get started by creating a table using parquet. Later, we will convert it to delta.

https://docs.delta.io/latest/delta-batch.html#id7

In [4]:
%%sql
CREATE TABLE delta101
USING parquet
PARTITIONED BY (month)
COMMENT 'Delta 101'
AS VALUES (0, 'zero', 1) t(id, name, month)

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

## ➡️ DESCRIBE EXTENDED

Next, validate whether the table has been registered in the lakehouse. Notice that the icon is a bit different (does not contain the triangle). The command below describes the current table

In [5]:
%%sql
DESC EXTENDED delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 7, Finished, Available)

<Spark SQL result set with 23 rows and 3 fields>

## ➡️ CONVERT TO DELTA

In [6]:
%%sql
CONVERT TO DELTA delta101
PARTITIONED BY (month INT)

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

Notice that with the above command, we still need to configure the partitioning field (again).

More information: [here](https://docs.delta.io/latest/delta-utility.html#convert-a-parquet-table-to-a-delta-table)


In [7]:
%%sql
DESCRIBE DETAIL delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 9, Finished, Available)

<Spark SQL result set with 1 rows and 14 fields>

## ➡️ DESCRIBE HISTORY

Provides provenance information, including the operation, user, and so on, and operation metrics for each write to a table. Table history is retained for 30 days.

More information at https://docs.delta.io/latest/delta-batch.html#-describe-history

In [8]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 10, Finished, Available)

<Spark SQL result set with 1 rows and 15 fields>

## ➡️ INSERT

In [9]:
%%sql
INSERT INTO delta101 VALUES (1, 'one', 1)

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 11, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
%%sql
SELECT * FROM delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 12, Finished, Available)

<Spark SQL result set with 2 rows and 3 fields>

In [11]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 13, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

## ➡️ UPDATE

In [12]:
%%sql

UPDATE delta101
SET name = 'one_UPDATE' WHERE id = 1

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 14, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [13]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 15, Finished, Available)

<Spark SQL result set with 3 rows and 15 fields>

## ➡️ DELETE

In [14]:
%%sql
DELETE FROM delta101 WHERE id > 0

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 16, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [15]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 17, Finished, Available)

<Spark SQL result set with 4 rows and 15 fields>

## ➡️ RESTORE


In [16]:
%%sql
RESTORE TABLE delta101
TO VERSION AS OF 2

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 18, Finished, Available)

<Spark SQL result set with 1 rows and 6 fields>

In [17]:
%%sql
SELECT * FROM delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 19, Finished, Available)

<Spark SQL result set with 2 rows and 3 fields>

In [18]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 20, Finished, Available)

<Spark SQL result set with 5 rows and 15 fields>

## ➡️ MERGE



In [19]:
%%sql
MERGE INTO delta101 AS target
USING (
    VALUES
        (1, 'not important', 1),
        (2, 'merge', 2) updates(id, name, month)
)
ON target.id = updates.id
WHEN MATCHED THEN
    UPDATE SET
        name = concat(target.name, '_merge')
WHEN NOT MATCHED THEN
    INSERT *

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 21, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [20]:
%%sql
SELECT * FROM delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 22, Finished, Available)

<Spark SQL result set with 3 rows and 3 fields>

In [21]:
%%sql
DESCRIBE HISTORY delta101

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 23, Finished, Available)

<Spark SQL result set with 6 rows and 15 fields>

## ➡️ TIME TRAVEL

In [22]:
%%spark
val q = spark.read.option("versionAsOf", 3).table("delta101")
display(q)

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, d0470ce7-47a8-4e13-8248-68e30a609e3a)


q: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]


In [23]:
%%sql
SELECT * FROM delta101
VERSION AS OF 5

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 25, Finished, Available)

<Spark SQL result set with 3 rows and 3 fields>

## ➡️ OPTIMIZE

As Delta tables change, performance and storage cost efficiency tend to degrade for the following reasons:

- New data added to the table might skew data
- Batch and streaming data ingestion rates might bring in many small files
- Update and delete operations eventually create read overhead; parquet files are immutable by design, so Delta tables adds new parquet files which the changeset, further amplifying the issues imposed by the first two items.
- No longer needed data files and log files available in the storage.

In order to keep the tables at the best state for best performance, perform bin-compaction and vacuuming operations in the Delta tables. Bin-compaction is achieved by the OPTIMIZE command; it merges all changes into bigger, consolidated parquet files.

The following command structures bin-compact and rewrite all affected files using V-Order

In [24]:
%%sql
OPTIMIZE delta101 VORDER

StatementMeta(, 1d736dfc-2ce9-4b93-a97b-3b9cde7d67f3, 26, Submitted, Running)

**V-Order is a write time optimization to the parquet file format** that enables lightning-fast reads under the Microsoft Fabric compute engines, such as Power BI, SQL, Spark and others.

Power BI and SQL engines make use of Microsoft Verti-Scan technology and V-Ordered parquet files to achieve in-memory like data access times. Spark and other non-Verti-Scan compute engines also benefit from the V-Ordered files with an average of 10% faster read times, with some scenarios up to 50%.

V-Order works by applying special sorting, row group distribution, dictionary encoding and compression on parquet files, thus requiring less network, disk, and CPU resources in compute engines to read it, providing cost efficiency and performance. V-Order sorting has a 15% impact on average write times but provides up to 50% more compression.

It's **100% open-source parquet format compliant**; all parquet engines can read it as a regular parquet files. Delta tables are more efficient than ever; features such as Z-Order are compatible with V-Order. Table properties and optimization commands can be used on control V-Order on its partitions.

V-Order is applied at the parquet file level. Delta tables and its features, such as Z-Order, compaction, vacuum, time travel, etc. are orthogonal to V-Order, as such, are compatible and can be used together for extra benefits.

More information https://learn.microsoft.com/en-us/fabric/data-engineering/delta-optimization-and-v-order?tabs=sparksql
